 ------------------------------------------------------------------------------------------------------------------
                                                  ABRHAM NEGASH GELAN
-----------------------------------------------------------------------------------------------------------------------
                                            CS-181 Final Project - Data Acquisition
                                                      08/11/22
--------------------------------------------------------------------------------------------------------------------

In [1]:
import os
import io
import sys
import importlib
import pandas as pd
from lxml import etree

**Central Question**: Using countries GDP, life expectancy, and literacy rate reports : “Does a better economic status indicate a better health status and education system in countries?”.

**Breakdown**: All these data sources have countries associated with them. I would create a larger database with countries that are represented in **all** data sets. In addition to that, the countries that will be added to the larger dataset are the ones that have values for gdp, life expectancy, and literacy rate. Then I will plot values in a graph to observe the relationship between gdp and life expectancy **and** gdp and literacy rate.

**Data Sources**:

DATA_SOURCE_1: List of countries by GDP https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_GDP_(nominal). Gross domestic product (GDP) is the annual market value of all final goods and services from a country. This data sources lists countries and their GDP in 2020.

DATA_SOURCE_2: List of countries by Life expectancy https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy. "Life expectancy is the average number of years a newborn would live if age-specific mortality rates in the current year were to stay the same throughout its life" Wikipedia. This data sources lists the life expectancy of countries recorded at 2022 by the United Nations 

DATA_SOURCE_3: List of countries by Literacy Rate https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_literacy_rate. Literacy rate is the estimate of percentages of population in a given country that can read and write. This data source lists countries that are members of the UN and their recently recorded literacy rate.

In [2]:
# Download the HTML to a file
!curl -s -o ds1.html https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_GDP_(nominal)
!curl -s -o ds2.html https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_life_expectancy
!curl -s -o ds3.html https://en.wikipedia.org/api/rest_v1/page/html/List_of_countries_by_literacy_rate

**Using etree to parse the root of the HTML tree from the downloaded html files**

In [3]:

htmlparser = etree.HTMLParser()

#DATA_SOURCE_1
path1 = os.path.join("ds1.html")
tree1 = etree.parse(path1, htmlparser)
ds1_root = tree1.getroot()

#DATA_SOURCE_2
path2 = os.path.join("ds2.html")
tree2 = etree.parse(path2, htmlparser)
ds2_root = tree2.getroot()

#DATA_SOURCE_3
path3 = os.path.join("ds3.html")
tree3 = etree.parse(path3, htmlparser)
ds3_root = tree3.getroot()


In [160]:
table_root1=ds1_root.xpath("//table[position()=1]/tbody")
#table_root1

#Columns
col_1=ds1_root.xpath("//table[position()=1]/tbody/tr[position()=1]/th[position()=1]/text()")
col_1=col_1+ds1_root.xpath("//table[position()=1]/tbody/tr[position()=2]/th[position()>4]/text()")

#Data

DoL={}
DoL[col_1[0]]=[]
DoL[col_1[1]]=[]

for i in range (4,220):
    c_region=ds1_root.xpath("//table[position()=1]/tbody/tr[position()='{}']/td[position()=1]/a/text()".format(i))
    estimate=ds1_root.xpath("//table[position()=1]/tbody/tr[position()='{}']/td[position()=7]/text()".format(i))
    DoL[col_1[0]].append(c_region[0])
    if estimate!=[]:
        DoL[col_1[1]].append(estimate[0])
    else:
        DoL[col_1[1]].append('N/A')


df1=pd.DataFrame(DoL)
#df1

**Method of Access**: xpath

**Accessing the table**: There are multiple tables in the page. Thus, I accessed the table I want by specifying its position in the body of the html.

**Columns**: The columns in this table were in different rows in the table so I accessed them based on their row position.

**Data**: initialized a dictionary of lists with the column names as keys and empty lists as values. Data values start from the fourth row and end at the last one, thus I accessed the appropriate values based on their positions. The country names were appended to the list with the country/regions key. For the estimate (gdp) values, an if condition was set to check if the values were empty. If they were, a string of 'N/A' was appended, if not, the values were appended.

**DataFrame creation**: A dataframe was created using the DoL

In [153]:
table_root2=ds2_root.xpath("//section[position()=4]/table/tbody")
#table_root2

#Columns
col_2=ds2_root.xpath("//section[position()=4]/table/tbody/tr[position()=1]/th/text()")

#Data
DoL2={}
DoL2[col_2[0]]=[]
DoL2[col_2[1]]=[]

for i in range (3,251):
    c_region2=ds2_root.xpath("//section[position()=4]/table/tbody/tr[position()='{}']/td[position()=1]/a/text()".format(i))
    life_exp=ds2_root.xpath("//section[position()=4]/table/tbody/tr[position()='{}']/td[position()=2]/text()".format(i))
    if c_region2!=[]:
        DoL2[col_2[0]].append(c_region2[0])
        DoL2[col_2[1]].append(life_exp[0])


df2=pd.DataFrame(DoL2)
#df2

**Method of Access**: xpath

**Accessing the table**: Accessed the desired table by specifying the section it was in.

**Columns**: First row in the body of the table

**Data**: initialized a dictionary of lists with the column names as keys and empty lists as values. Data values start from the third row and end at the last one, thus I accessed the appropriate values based on their positions.The first list also contains continents in addition to countries, which was irrelevant to the data desired. Therefore, and if condition was implemented to distinguish between country and continent values. Since the continent values were in a different position than that of the countries, the xpath would yield and empty list.

**DataFrame creation**: A dataframe was created using the DoL

In [154]:
table_root3=ds3_root.xpath("//section[position()=3]/table/tbody")
#table_root3

#Columns
col_3=ds3_root.xpath("//section[position()=3]/table/tbody/tr[position()=1]/th/text()")
col_3

#Data
countries=ds3_root.xpath("//section[position()=3]/table/tbody/tr[position()>2]/td[position()=1]/a/text()")
country_adj= [i.split('\u202f*')[0] for i in countries]
lit_rate=ds3_root.xpath("//section[position()=3]/table/tbody/tr[position()>2]/td[position()=2]/text()")


DoL3={}
DoL3[col_3[0]]=country_adj
DoL3[col_3[1]]=[lit_rate[i] for i in range (len(lit_rate)-1)]

df3=pd.DataFrame(DoL3)


**Method of Access**: xpath

**Accessing the table**: Accessed the desired table by specifying the section it was in.

**Columns**: Located in the first row in the body of the table

**Data**: Two different lists were returned from an xpath statement that accessed the value of countries and literacy rate. Values of countries came along with a string ('\u202f*') attached to their end which was removed using a list comprehension. Initialized a dictionary of lists with the columns as keys and the list of countries and literacy rates as values. The last value in the list of literacy rate was removed because it was the average literacy rate of the world (not a country)

**DataFrame creation**: A dataframe was created using the DoL

**Additional Information**

***Why these data sources? :*** This analysis would be interesting because GDP had always been critiqued for being an indicator that does not measure wealth of a country accurately. Analyzing the relationship between GDP **and** life expectancy and literacy rate would show if GDP plays a role in health status and education system in a country which could then be used to explain if GDP is still a good indicator for some variables other than wealth of a country.

***Terms of Use for Data Source Scrapping :*** https://wikimedia.org/api/rest_v1/

***Issues while scrapping :*** Implemented the method used for the third data source (create a list for every column and create a dataframe from those lists) for all data sources. However, some columns had missing values so the lists of columns had different lengths which did not allow for the creation of a dataframe. Another issue encountered was striping off the extra string for the country values in the third data source which was resolved after meeting with Dr. Amert.

In [ ]:
#Uncomment to preview parts of the dataframes

#df1.head(15)
#df2.head(15)
#df3.head(15)